# Solving Equilibria with Fixed Axis and Fixed NAE O(rho) Behavior in DESC

This tutorial shows how to find equilibrium solutions in DESC which are constrained to have the same axis and near-axis behavior as a NAE solution from the pyQSC code

# Creating a DESC Equilibrium from a pyQsc Near-Axis Equilibrium

Note that you must have pyQsc installed in order to make use of the `Equilibrium.from_near_axis` method, do so with `pip install qsc`

In [1]:
# must have installed pyQsc with `pip install qsc` in order to use this!
from qic import Qic
import numpy as np
import matplotlib.pyplot as plt
from desc.equilibrium import Equilibrium
from desc.objectives import get_fixed_boundary_constraints
from desc.plotting import (
    plot_comparison,
    plot_fsa,
    plot_section,
    plot_surfaces,
    plot_qs_error,
)

DESC version 0.9.2+587.gc0b44414,using JAX backend, jax version=0.4.13, jaxlib version=0.4.13, dtype=float64
Using device: CPU, with 14.58 GB available memory


DESC is able to create an equilibrium based off of a `pyQsc` NAE equilibrium object. First, we'll make the NAE equilibrium using `pyQsc`

In [2]:
qsc_eq = Qic.from_paper('QI NFP2 r2' , nphi=201,order="r2") # run with this one!
# qsc_eq.lasym=False

In [3]:
qsc_eq.lasym

True

Then, to make the DESC equilibrium, the `Equilibrium` class has a method `Equilibrium.from_near_axis`. This method creates a DESC `Equilibrium` based off of the `pyQsc` equilibrium. It requires as input the desired DESC FourierZernike resolution, as well as the radius at which you want to evaluate the qsc equilibrium at to make the DESC equilibrium's boundary. The equilibrium's initial `R_lmn`, `Z_lmn` Fourier-Zernike coefficients are fit to the `R,Z` evaluated from the `pyQsc` equilibrium, and the initial `L_lmn` are 0 (because the `pyQsc` equilibrium uses Boozer angles, so there is no poloidal stream function)

In [4]:
ntheta = 60
r = 0.01
desc_eq = Equilibrium.from_near_axis(
    qsc_eq,  # the Qsc equilibrium object
    r=r,  # the finite radius (m) at which to evaluate the Qsc surface to use as the DESC boundary
    L=7,  # DESC radial resolution
    M=7,  # DESC poloidal resolution
    N=9,  # DESC toroidal resolution
    ntheta=ntheta,
)
desc_eq.change_resolution(L_grid=12, M_grid=12, N_grid=15)
eq_fit = desc_eq.copy()  # copy so we can see the original Qsc surfaces later

In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
%pdb
from qic import Qic
import matplotlib.pyplot as plt
from desc.objectives.nae_utils import _calc_1st_order_NAE_coeffs
import numpy as np
# qsc = Qic.from_paper('QI NFP2 r2' , nphi=301,order="r1") # run with this one!
# qsc.lasym=False
# rc      = [1.0,0.0,-0.04989452393138029,0.0,0.005104315280094349,0.0,0.0003260666764535078]
# zs      = [0.0,0.0,-0.16444017885659307,0.0,0.005898705523953336,0.0,0.0011692891872228071]
# B0_vals = [1.0,0.15664582522744158]
# omn_method = "non-zone-fourier"
# k_buffer = 1
# p_buffer = 2
# d_over_curvature_cvals = [0.4523859311745555,0.0008679779426985831,-0.002484819813388363,0.0036775832032869262,0.013309425256703433,-0.00026299410476153604]
# delta   = 0.1
# d_svals = []
# nfp     = 3
# iota    = -2.1155667891936907
# X2s_svals = []
# X2c_cvals = [0.0]
# X2s_cvals = [0.0004066588899512191,0.06021894521908905,-0.4125633146872555,-0.0818377011695626,0.8415945963479472,0.18971830072842027,0.840558938779598,2.034624993033159,0.9352356722695293,1.6048050209546765,1.585922613305025,0.3859883515529046,0.26291440390429244,-0.48766371465964653,-1.1478558772733995,-1.0148962057914597,-1.058728731345739,0.016331648279707994,-0.4876961172005086,0.7474027188789079,0.4751643162543343,0.19548420651752554,0.5321064182261286,0.4511307390753144,0.1328582415680496,-0.11537707059274108,0.7234506414579371,1.139363120006459,0.09258286345339842,0.8860776402537537,0.7972098185125114,1.0589234885571917,0.004209462671819493,0.30320802234633165,1.5690263516269627,0.8501938303572988,0.45070833401041166,0.6157964001744047,-0.3870801525444373,-0.9915898964143361,0.18180959625909027,-0.14863572738786368,-0.8914080994267046,-1.6745490016858235,-0.6921642320133545,0.1593309546913136,1.560108303888641,-1.2027248590150141,0.6988908055525958,-0.27190159397058233,-0.0014903966202447932,-0.10437456640097813,-0.23282190841859718,-0.7706162363999018,1.95697384893869,0.256094408764415,-0.94967317708916,-1.3076316916144681,-1.0865590189725691,-0.26718857192560946,0.44453530342159997,-0.7098819766522084,-0.894994462752138,0.5626311905082779,0.3737435821405868,0.7746664490252777,1.4164570590540544,0.832061712027566,-0.2484649581729113,1.0222149095214341,0.5357447484163306,1.3073396851477357,0.38825340763715854,0.8563964487686905,1.6225476409724093,-0.33943062580410466,0.3464712265696533,0.9097561637745468,0.07446862537914248,0.15489720449686017,-0.07827084496674824,1.2327329280450074,-0.2575067387795579,-0.04618031541849188,-0.8051078772208673,-1.2177583764126414,-1.0089634247933637,-0.8975390869218834,0.16074245059110692,0.282887893566128,1.3075997765127094,1.6877919589007733,1.033465815337105,1.8676063446221485,1.4958552400916072,-0.11322155262523753,0.907022485213731,-0.5845985477131107,0.11424518226491291,-0.05356126020502161,-0.0015900406701586983]
# X2c_svals = [1.8881684958601035,4.121764860322693,-1.4333739576083322,0.6615556575938264,3.3300533681581017,5.649630257985376,7.773643724752436,9.29270472395089,10.736869853145656,11.316082414347825,10.253638057079307,7.8614122275506215,3.3013825843586297,-1.6978984994568203,-5.593793437042951,-7.004657470962415,-6.462537626440397,-4.730029282221211,-1.9420570513514877,0.9410699380528759,4.538435592695207,7.608941920943227,9.882618474937246,11.333482124676962,12.107914135012486,14.344592010869103,15.113866222825056,15.832217830235376,16.24599056367518,16.183532080566742,15.512423606912535,14.522724240145514,14.121813118186541,14.451317784056425,14.536769683080312,13.91823279363297,10.858283333161104,5.15203609868177,-1.469747016398436,-7.544727677193893,-11.080570240003802,-12.659978405752435,-12.751003171345811,-12.172070392937211,-11.586404122199955,-11.444829120609466,-12.645295408795656,-11.290752250995922,-8.9391709790351,-2.822232681428341,1.0662660502795824,-5.463875949392538,6.840567732140194,9.725098581831588,13.132363128532017,11.321703529266141,11.448695721409567,11.790335499496406,12.32753564579528,12.696242946418863,11.69159113257691,9.012469778736405,3.761142311693919,-2.97015219007647,-9.077759304254315,-13.230718207789119,-14.505119895161283,-14.431228463991491,-14.06763940609863,-14.158492165633644,-14.883546995654916,-16.08059711912246,-16.138649785073785,-15.934546378270621,-15.557143900275975,-14.62512099437439,-13.080510712454187,-11.688059866486995,-10.558821029138393,-8.686272946058928,-5.792683168226752,-2.205931736819775,0.9178956266453471,3.8433109802545724,6.017190465030275,6.89491202728108,6.386076175189963,3.227956853838462,-1.6061715068950062,-6.479411782819478,-9.76792668486508,-11.152202691696687,-10.961230648089451,-9.632349296637077,-8.320398299468057,-6.517942093883308,-4.270442842737996,-1.7547299946916184,1.294615618323907,-3.356234526443786,-0.5150801075313625]
# p2      = 0.0
# nphi=301
# order='r2'
# qsc = Qic(omn_method = omn_method, delta=delta, p_buffer=p_buffer, p2=p2, k_buffer=k_buffer, rc=rc,zs=zs, nfp=nfp, B0_vals=B0_vals, d_svals=d_svals, nphi=nphi, omn=True, B2c_cvals=X2c_cvals, B2s_svals=X2s_svals, order=order, d_over_curvature_cvals=d_over_curvature_cvals, B2c_svals=X2c_svals, B2s_cvals=X2s_cvals)

# coeffs,bases=_calc_1st_order_NAE_coeffs(qsc_eq,desc_eq)
# plt.rcParams.update({"font.size":18})

# coeffs.keys()

# coeffs['R_1_1_n']

# # bases.keys()

# plt.figure()
# plt.scatter(np.unique(modes["R_1_1_n_modes"]),np.abs(coeffs['R_1_1_n']))
# plt.yscale("log")
# plt.ylabel("R_1_1_n")
# plt.xlabel("n")
# plt.title("precise QI")

# plt.figure()
# plt.scatter(np.unique(modes["R_1_neg1_n_modes"]),np.abs(coeffs['R_1_neg1_n']))
# plt.yscale("log")
# plt.ylabel("R_1_neg1_n")
# plt.xlabel("n")
# plt.title("precise QI")

# plt.figure()
# plt.scatter(modes["Z_1_neg1_n_modes"],np.abs(coeffs['Z_1_neg1_n']))
# plt.yscale("log")
# plt.ylabel("Z_1_neg1_n")
# plt.xlabel("n")
# plt.title("precise QI")

# plt.figure()
# plt.scatter(modes["Z_1_1_n_modes"],np.abs(coeffs['Z_1_1_n']))
# plt.yscale("log")
# plt.ylabel("Z_1_1_n")
# plt.xlabel("n")
# plt.title("precise QI")


Automatic pdb calling has been turned ON


In [7]:
qsc_eq.lasym

True

Now we solve the equilibrium as normal in DESC

In [ ]:
# get the fixed-boundary constraints, which include also fixing the pressure and fixing the current profile (iota=False flag means fix current)
constraints = get_fixed_boundary_constraints(iota=False)
print(constraints)



# solve the equilibrium
desc_eq.solve(
    verbose=3,
    ftol=1e-2,
    objective="force",
    maxiter=100,
    xtol=1e-6,
    constraints=constraints,
)

# Save equilibrium as .h5 file
desc_eq.save(f"QIC_NFP2_N{desc_eq.N}_M{desc_eq.M}_L{desc_eq.L}_fixed_surf.h5")

/home/dpanici/DESC/desc/objectives/objective_funs.py:816: FutureWarning: Creating an Objective without specifying the Equilibrium to optimize is deprecated, in the future this will raise an error.
  warnings.warn(


(<desc.objectives.linear_objectives.FixBoundaryR object at 0x7f46158fdc40>, <desc.objectives.linear_objectives.FixBoundaryZ object at 0x7f46158fd910>, <desc.objectives.linear_objectives.FixPsi object at 0x7f46158fd7c0>, <desc.objectives.linear_objectives.FixPressure object at 0x7f46158fd8b0>, <desc.objectives.linear_objectives.FixCurrent object at 0x7f46158fd7f0>)
Building objective: force
Precomputing transforms
Timer: Precomputing transforms = 1.38 sec
Timer: Objective build = 4.23 sec
Timer: Linear constraint projection build = 10.3 sec
Compiling objective function and derivatives: ['force']
Timer: Objective compilation time = 2.27 sec
Timer: Jacobian compilation time = 10.5 sec
Timer: Total compilation time = 12.8 sec
Number of parameters: 1474
Number of objectives: 5642
Starting optimization
Using method: lsq-exact
   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1          5.143e-02                                   

Now we have a DESC equilibrium solved with the boundary from `pyQsc`. It has zero toroidal current as its profile constraint along with zero pressure since the original Qsc equilibrium had 0 pressure and current.

However, if we plot the surfaces, we see that while the boundary matches, the interior deviates slightly from the NAE, especially near the core. This means we may have lost some of the optimized properties from the QSC equilibrium.

In [ ]:
plot_comparison(
    eqs=[desc_eq, eq_fit],
    labels=["DESC-solved equilibrium", "NAE surfaces"],
    figsize=(12, 12),
    theta=0,
    colors=["k", "r"],
    linestyles=["-", ":"],
    lws=[1, 2],
);

Instead of evaluating the NAE at a finite radius, fixing that boundary, and hoping the axis behavior stays the same, we can instead directly fix the axis and the $O(\rho)$ asymptotic behavior.

# Solving Equilibria with Fixed Axis and Fixed NAE $O(\rho)$ Behavior in DESC

In [ ]:
# utility functions for getting the NAE constraints
from desc.objectives import get_equilibrium_objective, get_NAE_constraints

eq_NAE = eq_fit.copy()
# this has all the constraints we need, iota=False specifies we want to fix current instead of iota
constraints = get_NAE_constraints(eq_NAE, qsc_eq, iota=False, order=1)

eq_NAE.solve(
    verbose=3,
    ftol=1e-2,
    objective="force",
    maxiter=100,
    xtol=1e-6,
    constraints=constraints,
);
eq_NAE.save(f"QIC_NFP2_N{desc_eq.N}_M{desc_eq.M}_L{desc_eq.L}.h5")

In [ ]:
eq_NAE.sym

Again we can plot the surfaces, and we see that in this case, the near axis behavior is preserved, while the outer surfaces differ. This is expected, as the NAE from QSC is only valid asymptotically near the axis. By constraining this behavior we are able to keep all the desireable properties of the NAE where they are valid without overly constraining the problem.

In [ ]:
fig, ax = plot_comparison(
    eqs=[eq_fit, eq_NAE],
    labels=[f"NAE Surfaces r={r}", f"DESC NAE constrained"],
    colors=["k", "g"],
    linestyles=["-", "--"],
    figsize=(12, 12),
    theta=0,
    lws=[1, 2],
)
fig, ax = plot_comparison(
    eqs=[desc_eq, eq_NAE],
    labels=[f"DESC Fixed Surface", f"DESC NAE constrained"],
    colors=["r", "g"],
    linestyles=[":", "--"],
    figsize=(12, 12),
    theta=0,
    lws=[1, 2],
);

In [ ]:
fig, ax = plot_comparison(
    eqs=[eq_fit, eq_NAE],
    labels=[f"NAE Surfaces r={r}", f"DESC NAE constrained"],
    colors=["k", "g"],
    linestyles=["-", "--"],
    figsize=(12, 12),
    theta=0,
    lws=[1, 2],zeta=1.
)
fig, ax = plot_comparison(
    eqs=[desc_eq, eq_NAE],
    labels=[f"DESC Fixed Surface", f"DESC NAE constrained"],
    colors=["r", "g"],
    linestyles=[":", "--"],
    figsize=(12, 12),
    theta=0,
    lws=[1, 2],zeta=1.
);

As a final comparison, we can plot the QS error for the fixed boundary and fixed NAE solves. We see that by fixing the NAE behavior, we are able to preserve the QS from the original QSC equilibrium.

In [ ]:
fix, ax = plt.subplots()
plot_qs_error(
    desc_eq,
    fC=False,
    fT=False,
    log=True,
    ax=ax,
    colors=["r"],
    legend=False,
    labels=["Fixed boundary"],
)
plot_qs_error(
    eq_NAE,
    fC=False,
    fT=False,
    log=True,
    ax=ax,
    colors=["g"],
    legend=False,
    labels=["Fixed NAE"],
)
ax.legend()
ax.set_ylabel("$\sum |B_{mn}|$")

In [ ]:
from desc.plotting import *
from desc.grid import LinearGrid
plot_boozer_surface(eq_NAE)
plt.title(r"$\rho=1$")
plot_boozer_surface(eq_NAE,grid_plot = LinearGrid(rho=1e-5,M=40,N=40),grid_compute = LinearGrid(rho=1e-5,M=40,N=40))
plt.title(r"$\rho=10^-5$")

In [ ]:
plot_boozer_surface(desc_eq)
plt.title(r"$\rho=1$")
plot_boozer_surface(eq_NAE,grid_plot = LinearGrid(rho=1e-5,M=40,N=40),grid_compute = LinearGrid(rho=1e-5,M=40,N=40))
plt.title(r"$\rho=10^-5$")

The correct behaviour near the axis may be also checked by assessing directly other features of the equilibrium; most notably, the behaviour of |B| on axis, the rotational transform on axis, as well as the form of $\lambda$. Starting from the rotational transform, we may compare the value obtained to that of the near-axis expansion,

In [ ]:
import matplotlib.pyplot as plt
from desc.plotting import plot_fsa

rho = np.linspace(1e-3,1e-1)
fig, ax, iota_nae = plot_fsa(eq_NAE,"iota",rho=rho, return_data=True)
fig, ax, iota_surf = plot_fsa(desc_eq,"iota",rho=rho, ax=ax, return_data=True, linecolor='orange')
plt.plot(rho, np.ones(np.size(rho))*qsc_eq.iota,linestyle='--')
plt.legend(['Fixed NAE', 'Fixed boundary', 'NAE'])
print('Relative error in the rotational transform (fixed NAE): ', (iota_nae['iota'][0]-qsc_eq.iota)/qsc_eq.iota)
print('Relative error in the rotational transform (fixed surface): ', (iota_surf['iota'][0]-qsc_eq.iota)/qsc_eq.iota)
# print(iota[1].yaxis)

In [ ]:
import matplotlib.pyplot as plt
from desc.plotting import plot_fsa

rho = np.linspace(1e-3,1,50)
fig, ax, iota_nae = plot_fsa(eq_NAE,"iota",rho=rho, return_data=True)
fig, ax, iota_surf = plot_fsa(desc_eq,"iota",rho=rho, ax=ax, return_data=True, linecolor='orange')
plt.plot(rho, np.ones(np.size(rho))*qsc_eq.iota,linestyle='--')
plt.legend(['Fixed NAE', 'Fixed boundary', 'NAE'])
print('Relative error in the rotational transform (fixed NAE): ', (iota_nae['iota'][0]-qsc_eq.iota)/qsc_eq.iota)
print('Relative error in the rotational transform (fixed surface): ', (iota_surf['iota'][0]-qsc_eq.iota)/qsc_eq.iota)
# print(iota[1].yaxis)

This clearly shows that the constraint is working as it is intended. A similar expected behaviour can be found for other features of the equilibrium such as the magnetic field on axis.

In [ ]:
from desc.compute import data_index
from desc.grid import LinearGrid

grid = LinearGrid(M=desc_eq.M_grid, N=desc_eq.N_grid, NFP=desc_eq.NFP, rho=np.array(1e-6))
# Evaluate B modes near the axis
data_surf = desc_eq.compute(["|B|_mn", "B modes"], grid=grid)
data_nae = eq_NAE.compute(["|B|_mn", "B modes"], grid=grid)
modes = data_surf["B modes"]
B_mn_surf = data_surf["|B|_mn"]
B_mn_nae = data_nae["|B|_mn"]
# Evaluate B on an angular grid
theta = np.linspace(0,2*np.pi,150)
phi = np.linspace(0,2*np.pi,qsc_eq.nphi)
th, ph = np.meshgrid(theta,phi)
B_surf = np.zeros((qsc_eq.nphi,150))
B_nae = np.zeros((qsc_eq.nphi,150))
idx = np.where(modes[:,2] !=0)[0]
# Print the deviation of B from QS (that is, the variation of B respect to its average along the axis)
print('Deviation from QS (fixed surface): ', np.sqrt(np.sum(B_mn_surf[idx]*B_mn_surf[idx]))/np.sqrt(np.sum(B_mn_surf*B_mn_surf)))
print('Deviation from QS (fixed nae): ', np.sqrt(np.sum(B_mn_nae[idx]*B_mn_nae[idx]))/np.sqrt(np.sum(B_mn_nae*B_mn_nae)))
for i, (l,m,n) in enumerate(modes):
    if m>=0 and n>=0:
        B_surf += B_mn_surf[i]*np.cos(m*th)*np.cos(n*ph)
        B_nae += B_mn_nae[i]*np.cos(m*th)*np.cos(n*ph)
    elif m>=0 and n<0:
        B_surf += -B_mn_surf[i]*np.cos(m*th)*np.sin(n*ph)
        B_nae += -B_mn_nae[i]*np.cos(m*th)*np.sin(n*ph)
    elif m<0 and n>=0:
        B_surf += -B_mn_surf[i]*np.sin(m*th)*np.cos(n*ph)
        B_nae += -B_mn_nae[i]*np.sin(m*th)*np.cos(n*ph)
    elif m<0 and n<0:
        B_surf += B_mn_surf[i]*np.sin(m*th)*np.sin(n*ph)
        B_nae += B_mn_nae[i]*np.sin(m*th)*np.sin(n*ph)
# Eliminate the poloidal angle to focus on the toroidal behaviour
B_av_surf = np.mean(B_surf,axis=1)
B_av_nae = np.mean(B_nae,axis=1)
plt.plot(phi,B_av_surf)
plt.plot(phi,B_av_nae)
plt.plot(phi,np.ones(np.size(phi))*qsc_eq.B0,linestyle='--')
plt.xlabel('$\phi$')
plt.ylabel('$|B|$')
plt.legend(['Fixed surface', 'Fixed NAE','NAE'])
plt.show()

In [ ]:
from desc.compute import data_index
from desc.grid import LinearGrid

grid = LinearGrid(M=desc_eq.M_grid, N=desc_eq.N_grid, NFP=desc_eq.NFP, rho=np.array(1e-9))
# Evaluate B modes near the axis
data_nae = eq_NAE.compute(["|B|_mn", "B modes"], grid=grid)
modes = data_nae["B modes"]
B_mn_nae = data_nae["|B|_mn"]
# Evaluate B on an angular grid
theta = np.linspace(0,2*np.pi,150)
phi = np.linspace(0,2*np.pi,qsc_eq.nphi)
th, ph = np.meshgrid(theta,phi)
B_nae = np.zeros((qsc_eq.nphi,150))
idx = np.where(modes[:,2] !=0)[0]
# Print the deviation of B from QS (that is, the variation of B respect to its average along the axis)
print('Deviation from QS (fixed nae): ', np.sqrt(np.sum(B_mn_nae[idx]*B_mn_nae[idx]))/np.sqrt(np.sum(B_mn_nae*B_mn_nae)))
for i, (l,m,n) in enumerate(modes):
    if m>=0 and n>=0:
        B_nae += B_mn_nae[i]*np.cos(m*th)*np.cos(n*ph)
    elif m>=0 and n<0:
        B_nae += -B_mn_nae[i]*np.cos(m*th)*np.sin(n*ph)
    elif m<0 and n>=0:
        B_nae += -B_mn_nae[i]*np.sin(m*th)*np.cos(n*ph)
    elif m<0 and n<0:
        B_nae += B_mn_nae[i]*np.sin(m*th)*np.sin(n*ph)
# Eliminate the poloidal angle to focus on the toroidal behaviour
B_av_nae = np.mean(B_nae,axis=1)
np.testing.assert_allclose(B_av_nae, np.ones(np.size(phi))*qsc_eq.B0, atol=2e-2)



plt.plot(phi,B_av_nae)
plt.plot(phi,np.ones(np.size(phi))*qsc_eq.B0,linestyle='--')
plt.xlabel('$\phi$')
plt.ylabel('$|B|$')
plt.legend(['Fixed surface', 'Fixed NAE','NAE'])
plt.show()

In [ ]:
from desc.compute import data_index
from desc.grid import LinearGrid
from desc.io import load
for N in [19,15,11]:
    eq_NAE = load(f"QIC_NFP2_N{N}_M7_L7.h5")
    # eq_surf = load(f"QIC_NFP2_N{N}_M7_L7_fixed_surf.h5")
    # eq_surf.change_resolution(eq_NAE)
    grid = LinearGrid(M=eq_NAE.M_grid, N=eq_NAE.N_grid, NFP=eq_NAE.NFP, rho=np.array(1e-6))
    # Evaluate B modes near the axis
    # data_surf = desc_eq.compute(["|B|_mn", "B modes"], grid=grid)
    data_nae = eq_NAE.compute(["|B|_mn", "B modes"], grid=grid)
    modes = data_nae["B modes"]
    # B_mn_surf = data_surf["|B|_mn"]
    B_mn_nae = data_nae["|B|_mn"]
    # Evaluate B on an angular grid
    theta = np.linspace(0,2*np.pi,150)
    phi = np.linspace(0,2*np.pi,qsc_eq.nphi)
    th, ph = np.meshgrid(theta,phi)
    B_surf = np.zeros((qsc_eq.nphi,150))
    B_nae = np.zeros((qsc_eq.nphi,150))
    idx = np.where(modes[:,2] !=0)[0]
    # Print the deviation of B from QS (that is, the variation of B respect to its average along the axis)
    # print('Deviation from QS (fixed surface): ', np.sqrt(np.sum(B_mn_surf[idx]*B_mn_surf[idx]))/np.sqrt(np.sum(B_mn_surf*B_mn_surf)))
    print('Deviation from QS (fixed nae): ', np.sqrt(np.sum(B_mn_nae[idx]*B_mn_nae[idx]))/np.sqrt(np.sum(B_mn_nae*B_mn_nae)))
    for i, (l,m,n) in enumerate(modes):
        if m>=0 and n>=0:
            # B_surf += B_mn_surf[i]*np.cos(m*th)*np.cos(n*ph)
            B_nae += B_mn_nae[i]*np.cos(m*th)*np.cos(n*ph)
        elif m>=0 and n<0:
            # B_surf += -B_mn_surf[i]*np.cos(m*th)*np.sin(n*ph)
            B_nae += -B_mn_nae[i]*np.cos(m*th)*np.sin(n*ph)
        elif m<0 and n>=0:
            # B_surf += -B_mn_surf[i]*np.sin(m*th)*np.cos(n*ph)
            B_nae += -B_mn_nae[i]*np.sin(m*th)*np.cos(n*ph)
        elif m<0 and n<0:
            # B_surf += B_mn_surf[i]*np.sin(m*th)*np.sin(n*ph)
            B_nae += B_mn_nae[i]*np.sin(m*th)*np.sin(n*ph)
    # Eliminate the poloidal angle to focus on the toroidal behaviour
    # B_av_surf = np.mean(B_surf,axis=1)
    B_av_nae = np.mean(B_nae,axis=1)
    # plt.plot(phi,B_av_surf)
    plt.plot(phi,B_av_nae,label=f"N={N}")
plt.plot(phi,np.ones(np.size(phi))*qsc_eq.B0,linestyle='--',label="NAE")
plt.xlabel('$\phi$')
plt.ylabel('$|B|$')
# plt.legend(['Fixed surface', 'Fixed NAE','NAE'])
plt.legend()
plt.show()


Finally, we may check $\lambda$ on axis and compare it to $\nu$ from within the near-axis expansion.

In [ ]:
grid_2d_05 = LinearGrid(rho=np.array(1e-6), M=50, N=50, NFP=desc_eq.NFP, endpoint=True)

# Evaluate lambda near the axis
data_surf = desc_eq.compute("lambda", grid=grid_2d_05)
data_nae = eq_NAE.compute("lambda", grid=grid_2d_05)
lam_surf = data_surf["lambda"]
lam_nae = data_nae["lambda"]

# Reshape to form grids on theta and phi
zeta = (
    grid_2d_05.nodes[:, 2]
    .reshape((grid_2d_05.num_theta, grid_2d_05.num_rho, grid_2d_05.num_zeta), order="F")
    .squeeze()
)
lam_surf = lam_surf.reshape(
    (grid_2d_05.num_theta, grid_2d_05.num_rho, grid_2d_05.num_zeta), order="F"
)
lam_nae = lam_nae.reshape(
    (grid_2d_05.num_theta, grid_2d_05.num_rho, grid_2d_05.num_zeta), order="F"
)

phi = np.squeeze(zeta[0, :])
lam_surf = np.squeeze(lam_surf[:, 0, :])
lam_nae = np.squeeze(lam_nae[:, 0, :])

# Eliminate the poloidal angle to focus on the toroidal behaviour
lam_av_surf = np.mean(lam_surf,axis=0)
lam_av_nae = np.mean(lam_nae,axis=0)
print('Deviation of theta from Boozer angle (fixed surface)', np.mean(np.abs(lam_av_surf+qsc_eq.iota*qsc_eq.nu_spline(phi))))
print('Deviation of theta from Boozer angle (fixed nae)', np.mean(np.abs(lam_av_nae+qsc_eq.iota*qsc_eq.nu_spline(phi))))
plt.plot(phi,lam_av_surf)
plt.plot(phi,lam_av_nae)
plt.plot(phi,-qsc_eq.iota*qsc_eq.nu_spline(phi),linestyle='--')
plt.xlabel('$\phi$')
plt.ylabel('$\lambda$')
plt.legend(['Fixed surface', 'Fixed NAE','NAE'])
plt.show()

In [ ]:
grid_2d_05 = LinearGrid(rho=np.array(1e-6), M=50, N=50, NFP=eq_NAE.NFP, endpoint=True)

# Evaluate lambda near the axis
data_nae = eq_NAE.compute("lambda", grid=grid_2d_05)
lam_surf = data_surf["lambda"]
lam_nae = data_nae["lambda"]

# Reshape to form grids on theta and phi
zeta = (
    grid_2d_05.nodes[:, 2]
    .reshape((grid_2d_05.num_theta, grid_2d_05.num_rho, grid_2d_05.num_zeta), order="F")
    .squeeze()
)

lam_nae = lam_nae.reshape(
    (grid_2d_05.num_theta, grid_2d_05.num_rho, grid_2d_05.num_zeta), order="F"
)

phi = np.squeeze(zeta[0, :])
lam_nae = np.squeeze(lam_nae[:, 0, :])

lam_qsc = -qsc_eq.iota*qsc_eq.nu_spline(phi)
lam_av_nae = np.mean(lam_nae,axis=0)
np.testing.assert_allclose(lam_av_nae, -qsc_eq.iota*qsc_eq.nu_spline(phi), atol=4e-5)

# Eliminate the poloidal angle to focus on the toroidal behaviour
lam_av_nae = np.mean(lam_nae,axis=0)
print('Deviation of theta from Boozer angle (fixed nae)', np.mean(np.abs(lam_av_nae+qsc_eq.iota*qsc_eq.nu_spline(phi))))
plt.plot(phi,lam_av_nae)
plt.plot(phi,-qsc_eq.iota*qsc_eq.nu_spline(phi),linestyle='--')
plt.xlabel('$\phi$')
plt.ylabel('$\lambda$')
plt.legend(['Fixed surface', 'Fixed NAE','NAE'])
plt.show()

The above comparison shows that the DESC equilibrium constructed with the near-axis constraint is consistent with the near-axis behaviour.

In [ ]:
# eq_NAE.save(f"QIC_NFP2_N{desc_eq.N}_M{desc_eq.M}_L{desc_eq.L}_r{r}.h5")

In [ ]:
1

In [ ]:
from desc.compute.utils import compress

In [ ]:
import os

import numpy as np
from shapely.geometry import Polygon

from desc.grid import Grid, LinearGrid
from desc.vmec import VMECIO

def area_difference(Rr1, Rr2, Zr1, Zr2, Rv1, Rv2, Zv1, Zv2):
    """Compute area difference between coordinate curves.

    Parameters
    ----------
    args : ndarray
        R and Z coordinates of constant rho (r) or vartheta (v) contours.
        Arrays should be indexed as [rho,theta,zeta]

    Returns
    -------
    area_rho : ndarray, shape(Nz, Nr)
        normalized area difference of rho contours, computed as the symmetric
        difference divided by the intersection
    area_theta : ndarray, shape(Nt, Nz)
        normalized area difference between vartheta contours, computed as the area
        of the polygon created by closing the two vartheta contours divided by the
        perimeter squared
    """
    assert Rr1.shape == Rr2.shape == Zr1.shape == Zr2.shape
    assert Rv1.shape == Rv2.shape == Zv1.shape == Zv2.shape

    poly_r1 = np.array(
        [
            [Polygon(np.array([R, Z]).T) for R, Z in zip(Rr1[:, :, i], Zr1[:, :, i])]
            for i in range(Rr1.shape[2])
        ]
    )
    poly_r2 = np.array(
        [
            [Polygon(np.array([R, Z]).T) for R, Z in zip(Rr2[:, :, i], Zr2[:, :, i])]
            for i in range(Rr2.shape[2])
        ]
    )
    poly_v = np.array(
        [
            [
                Polygon(np.array([R, Z]).T)
                for R, Z in zip(
                    np.hstack([Rv1[:, :, i].T, Rv2[::-1, :, i].T]),
                    np.hstack([Zv1[:, :, i].T, Zv2[::-1, :, i].T]),
                )
            ]
            for i in range(Rv1.shape[2])
        ]
    )

    diff_rho = np.array(
        [
            poly1.symmetric_difference(poly2).area
            for poly1, poly2 in zip(poly_r1.flat, poly_r2.flat)
        ]
    ).reshape((Rr1.shape[2], Rr1.shape[0]))
    intersect_rho = np.array(
        [
            poly1.intersection(poly2).area
            for poly1, poly2 in zip(poly_r1.flat, poly_r2.flat)
        ]
    ).reshape((Rr1.shape[2], Rr1.shape[0]))
    area_rho = np.where(
        diff_rho > 0, diff_rho / np.where(intersect_rho != 0, intersect_rho, 1), 0
    )
    area_theta = np.array(
        [poly.area / (poly.length) ** 2 for poly in poly_v.flat]
    ).reshape((Rv1.shape[1], Rv1.shape[2]))
    return area_rho, area_theta
def area_difference_desc(eq1, eq2, Nr=10, Nt=8, Nz=None):
    """Compute average normalized area difference between two DESC equilibria.

    Parameters
    ----------
    eq1, eq2 : Equilibrium
        desc equilibria to compare
    Nr : int, optional
        Number of radial surfaces to average over
    Nt : int, optional
        Number of vartheta contours to compare
    Nz : int, optional
        Number of zeta planes to compare. If None, use 1 plane for axisymmetric cases
        or 6 for non-axisymmetric.

    Returns
    -------
    area_rho : ndarray, shape(Nr, Nz)
        normalized area difference of rho contours, computed as the symmetric
        difference divided by the intersection
    area_theta : ndarray, shape(Nt, Nz)
        normalized area difference between vartheta contours, computed as the area
        of the polygon created by closing the two vartheta contours divided by the
        perimeter squared

    """
    Rr1, Zr1, Rv1, Zv1 = compute_coords(eq1, Nr=Nr, Nt=Nt, Nz=Nz)
    Rr2, Zr2, Rv2, Zv2 = compute_coords(eq2, Nr=Nr, Nt=Nt, Nz=Nz)

    area_rho, area_theta = area_difference(Rr1, Rr2, Zr1, Zr2, Rv1, Rv2, Zv1, Zv2)
    return area_rho, area_theta

def compute_coords(equil, Nr=10, Nt=8, Nz=None):
    """Computes coordinate values from a given equilibrium."""
    if Nz is None and equil.N == 0:
        Nz = 1
    elif Nz is None:
        Nz = 6

    num_theta = 1000
    num_rho = 1000

    # flux surfaces to plot
    rr = np.linspace(1, 0, Nr, endpoint=False)[::-1]
    rt = np.linspace(0, 2 * np.pi, num_theta)
    rz = np.linspace(0, 2 * np.pi / equil.NFP, Nz, endpoint=False)
    r_grid = LinearGrid(rho=rr, theta=rt, zeta=rz, NFP=equil.NFP)

    # straight field-line angles to plot
    tr = np.linspace(0, 1, num_rho)
    tt = np.linspace(0, 2 * np.pi, Nt, endpoint=False)
    tz = np.linspace(0, 2 * np.pi / equil.NFP, Nz, endpoint=False)
    t_grid = LinearGrid(rho=tr, theta=tt, zeta=tz, NFP=equil.NFP)

    # Note: theta* (also known as vartheta) is the poloidal straight field-line
    # angle in PEST-like flux coordinates

    # find theta angles corresponding to desired theta* angles
    v_grid = Grid(equil.compute_theta_coords(t_grid.nodes))
    r_coords = equil.compute(["R", "Z"], grid=r_grid)
    v_coords = equil.compute(["R", "Z"], grid=v_grid)

    # rho contours
    Rr1 = r_coords["R"].reshape(
        (r_grid.num_theta, r_grid.num_rho, r_grid.num_zeta), order="F"
    )
    Rr1 = np.swapaxes(Rr1, 0, 1)
    Zr1 = r_coords["Z"].reshape(
        (r_grid.num_theta, r_grid.num_rho, r_grid.num_zeta), order="F"
    )
    Zr1 = np.swapaxes(Zr1, 0, 1)

    # vartheta contours
    Rv1 = v_coords["R"].reshape(
        (t_grid.num_theta, t_grid.num_rho, t_grid.num_zeta), order="F"
    )
    Rv1 = np.swapaxes(Rv1, 0, 1)
    Zv1 = v_coords["Z"].reshape(
        (t_grid.num_theta, t_grid.num_rho, t_grid.num_zeta), order="F"
    )
    Zv1 = np.swapaxes(Zv1, 0, 1)

    return Rr1, Zr1, Rv1, Zv1

In [ ]:

# qsc = Qic.from_paper("QI Jorge",nphi=301)
qsc = qsc_eq
# ntheta = 75
# r = 0.01
# N = 9
# eq = Equilibrium.from_near_axis(qsc, r=r, L=6, M=6, N=N, ntheta=ntheta)
eq = eq_NAE

orig_Rax_val = eq_fit.axis.R_n
orig_Zax_val = eq_fit.axis.Z_n

# eq_fit = eq.copy()

# this has all the constraints we need,
#  iota=False specifies we want to fix current instead of iota
cs = get_NAE_constraints(eq, qsc, iota=False, order=1)

# objectives = ForceBalance()
# obj = ObjectiveFunction(objectives)

# eq.solve(verbose=3, ftol=1e-2, objective=obj, maxiter=50, xtol=1e-6, constraints=cs)

# Make sure axis is same
np.testing.assert_almost_equal(orig_Rax_val, eq.axis.R_n)
np.testing.assert_almost_equal(orig_Zax_val, eq.axis.Z_n)

# Make sure surfaces of solved equilibrium are similar near axis as QIC
rho_err, theta_err = area_difference_desc(eq, eq_fit)

np.testing.assert_allclose(rho_err[:, 0:-6], 0, atol=1e-2)
np.testing.assert_allclose(theta_err[:, 0:-6], 0, atol=1e-3)

# Make sure iota of solved equilibrium is same near axis as QIC
grid = LinearGrid(L=10, M=20, N=20, sym=True, axis=False)
iota = compress(grid, eq.compute("iota", grid=grid)["iota"], "rho")

np.testing.assert_allclose(iota[1], qsc.iota, atol=2e-4)
np.testing.assert_allclose(iota[1:10], qsc.iota, atol=1e-3)